modified from https://huggingface.co/agents-course/notebooks/blob/main/unit2/llama-index/agents.ipynb

In [1]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.core.agent.workflow import AgentStream, AgentWorkflow, ToolCallResult
from llama_index.core.tools import QueryEngineTool
from llama_index.core.workflow import Context
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.vector_stores.chroma import ChromaVectorStore


In [2]:
llm = GoogleGenAI(
    model="models/gemini-2.0-flash",
    # api_key="some key",  # uses GOOGLE_API_KEY env var by default
)
embed_model = GoogleGenAIEmbedding(model_name="text-embedding-004")

# Initialising agents

In [ ]:
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers"""
    return a - b


def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


def divide(a: int, b: int) -> int:
    """Divide two numbers"""
    return a / b


agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[subtract, multiply, divide, add],
    llm=llm,
    system_prompt="You are a math agent that can add, subtract, multiply, and divide numbers using provided tools.",
)

In [ ]:
handler = agent.run("What is (2 + 2) * 2?")
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp

In [ ]:
ctx = Context(agent)

response = await agent.run("My name is Bob.", ctx=ctx)
response = await agent.run("What was my name again?", ctx=ctx)
response

# Creating RAG Agents with QueryEngineTools

In [3]:
# Create a vector store
db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)
query_engine = index.as_query_engine(llm=llm)
query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="personas",
    description="descriptions for various types of personas",
    return_direct=False,
)

# Create a RAG agent
query_engine_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[query_engine_tool],
    llm=llm,
    system_prompt="You are a helpful assistant that has access to a database containing persona descriptions. ",
)

In [4]:
handler = query_engine_agent.run(
    "Search the database for 'science fiction' and return some persona descriptions."
)
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp


Called tool:  personas {'input': 'science fiction'} => The provided text does not contain information about science fiction.

I'm sorry, I couldn't find any persona descriptions related to "science fiction" in the database.

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='I\'m sorry, I couldn\'t find any persona descriptions related to "science fiction" in the database.')]), tool_calls=[ToolCallResult(tool_name='personas', tool_kwargs={'input': 'science fiction'}, tool_id='personas', tool_output=ToolOutput(content='The provided text does not contain information about science fiction.\n', tool_name='personas', raw_input={'input': 'science fiction'}, raw_output=Response(response='The provided text does not contain information about science fiction.\n', source_nodes=[NodeWithScore(node=TextNode(id_='cf474147-104a-4d21-ab10-5d698e2e5fd2', embedding=None, metadata={'file_path': '/home/zihaog/repos/Coral_de_LLM_recipes/Hugging_Face_Agents_Course/unit2/llama-index/data/persona_1002.txt', 'file_name': 'persona_1002.txt', 'file_type': 'text/plain', 'file_size': 122, 'creation_date': '2025-03-25', 'last_modif